<b>Pertemuan 6 Lab PML // SAMPLE</b> <br>
&#169; Gary Alvaro

<b style="color: red">Pastikan teman-teman sudah coba menjalankan file ini sebelum praktikum dimulai, dan pastikan semua libraries sudah terinstall. </b>

Silahkan download file di bawah dan letakkan di folder/direktori yang sama dengan file notebook ini.

<a href="https://drive.google.com/file/d/126WDCZMZNujwRaxAs-Y9AvvoSUDfcIX3/view?usp=share_link" target="_blank">https://drive.google.com/file/d/126WDCZMZNujwRaxAs-Y9AvvoSUDfcIX3/view?usp=share_link</a>

## Import Libraries yang dibutuhkan

In [22]:
# Import Pandas
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'

# Import Numpy
import numpy as np

# Loading Bar TQDM
from tqdm import tqdm

# Stopword Removal
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.tokenize import word_tokenize

# Stemming (Sastrawi)
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

# Tokenizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

# Pickle FastText
import pickle

# Split Data
from sklearn.model_selection import train_test_split

# Model Building
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dropout, LSTM, Dense
from tensorflow.keras.backend import clear_session
from keras.models import load_model

# Callbacks
from keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import EarlyStopping

# Grafik
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("whitegrid")
np.random.seed(0)

[nltk_data] Downloading package stopwords to C:\Users\GARY
[nltk_data]     ALVARO\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Mendefinisikan Variabel Penting

In [2]:
testSize = 0.15 #Pembagian ukuran datatesing

MAX_NB_WORDS = 100000 #Maximum jumlah kata pada vocabulary yang akan dibuat
max_seq_len = 46 #Panjang kalimat maximum

num_epochs = 40 #Jumlah perulangan / epoch saat proses training

aspek_kategori = ['akurasi', 'kualitas', 'pelayanan', 'pengemasan', 'harga', 'pengiriman']

## Read & Split Data

In [3]:
def read_data(filename):
    # Membaca file CSV ke dalam Dataframe
    df = pd.read_csv(filename) 
    
    # Encoding Categorical Data (Mengubah data kategorikal menjadi angka)
    label_dict = {'negatif':0, 'positif':1, '-':99}
    df['s_akurasi'] = df['s_akurasi'].replace(label_dict)
    df['s_kualitas'] = df['s_kualitas'].replace(label_dict)
    df['s_pelayanan'] = df['s_pelayanan'].replace(label_dict)
    df['s_pengemasan'] = df['s_pengemasan'].replace(label_dict)
    df['s_harga'] = df['s_harga'].replace(label_dict)
    df['s_pengiriman'] = df['s_pengiriman'].replace(label_dict)    
    
    # Membagi dataframe menjadi data training & testing
    df_training, df_testing = train_test_split(df, test_size=0.2, random_state=42, shuffle=True)

    # Reset Index
    df_training = df_training.reset_index()
    df_testing = df_testing.reset_index()

    return df_training, df_testing

## Preprocessing

Kali ini, preprocessing akan dibuat dalam bentuk singkat menggunakan fungsi.

Lakukan pendefenisian fungsi dan pemanggilan fungsi.

In [4]:
def preprocessing(data):
    # Case Folding
    data['lower'] = data['teks'].str.lower()
    
    # Punctual Removal
    data['punctual'] = data['lower'].str.replace('[^a-zA-Z]+',' ', regex=True)
    
    # Normalization
    kamus_baku = pd.read_csv('../Pertemuan 4 - Preprocessing NLP/kata_baku.csv', sep=";")
    dict_kamus_baku = kamus_baku[['slang','baku']].to_dict('list')
    dict_kamus_baku = dict(zip(dict_kamus_baku['slang'], dict_kamus_baku['baku']))
    norm = []
    for i in data['punctual']:
        res = " ".join(dict_kamus_baku.get(x, x) for x in str(i).split())
        norm.append(str(res))
    data['normalize'] = norm
    
    # Stopword Removal
    stop_words = set(stopwords.words('indonesian'))
    swr = []
    for i in tqdm(data['normalize']):
        tokens = word_tokenize(i)
        filtered = [word for word in tokens if word not in stop_words]
        swr.append(" ".join(filtered))
    data['stopwords'] = swr
    
    # Stemming
    factory = StemmerFactory()
    stemmer = factory.create_stemmer()
    stem = []
    for i in tqdm(data['stopwords']):
        stem.append(stemmer.stem(str(i)))
    data['stemmed'] = stem
    
    return data

In [5]:
# Pemanggilan fungsi read_data

filename = "../Pertemuan 4 - Preprocessing NLP/DatasetReviewProduk-LabPML.csv"
df_training, df_testing = read_data(filename)

In [7]:
# Pemanggilan fungsi preprocessing

df_training = preprocessing(df_training)
df_testing = preprocessing(df_testing)

100%|████████████████████████████████████████████████████████████████████████████████| 300/300 [00:24<00:00, 12.32it/s]


In [8]:
# Mengecek panjang kalimat maksimum

longest_string = max(df_training['stemmed'].values.tolist(), key=len)
print(len(longest_string.split()))

print(longest_string)

46
kali komplain kali kasih barang rusak driver grab salah kirim barang nunggu kirim komplain suruh grab kembali barang salah salah lapak diskusi komplain server error tramsaksi bl selesai dana terus lapak transaksi masalah komplain chat pelapam kirim ganti barang tes rusak komplain kali rusak terima kecewa


## Tokenisasi

In [9]:
# Tokenisasi Training
def tokenize_training(data_training):
    global tokenizer #Menggunakan variabel global agar 'tokenizer' bisa dipake di luar fungsi ini
    tokenizer = Tokenizer(num_words = MAX_NB_WORDS, char_level=False)
    tokenizer.fit_on_texts(data_training['stemmed'])
    word_index = tokenizer.word_index
    
    train_sequences = tokenizer.texts_to_sequences(data_training['stemmed'])
    word_seq_train = pad_sequences(train_sequences, maxlen = max_seq_len)

    return word_index, word_seq_train


# Tokenisasi Testing
def tokenize_testing(data_testing):
    test_sequences = tokenizer.texts_to_sequences(data_testing['stemmed'])
    word_seq_test = pad_sequences(test_sequences, maxlen = max_seq_len)

    return word_seq_test

In [10]:
# Tokenisasi secara umum
def tokenize(data):
    sequences = tokenizer.texts_to_sequences(data['stemmed'])
    word_seq = pad_sequences(sequences, maxlen = max_seq_len)

    return word_seq

In [11]:
# Pemanggilan Fungsi Tokenisasi

word_index, word_seq_train = tokenize_training(df_training)
word_seq_test = tokenize_testing(df_testing)